## Generate synthetic data [needs work in course selection algortihm]
## convert text to numeric using tfidf [not used currently]

In [19]:
import pandas as pd
pd.set_option('display.max_columns', None) 
from os import listdir
import os
import glob
import spacy
from collections import Counter
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy
from spacy.training import Example
from spacy.util import minibatch, compounding
from sklearn.model_selection import train_test_split
from spacy.scorer import Scorer
import re
import nlp
#https://chat.openai.com/share/c141bbaf-9fa9-41df-b29a-68653cfafd48

[nltk_data] Downloading package stopwords to /Users/nyzy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [67]:
df_c_cleaned = pd.read_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/course_cleaned.csv")
df_j_cleaned=pd.read_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/job_cleaned.csv")
df_s_skills_list = pd.read_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/skills_dataset/skills_list_df.csv")

skills_list = df_s_skills_list["skills_list"].unique().tolist()

In [68]:
df_j_cleaned = df_j_cleaned[df_j_cleaned["title_cleaned"]!="unmatched"]

In [84]:
len(df_j_cleaned["title_cleaned"].unique())

56

In [85]:
df_j_cleaned["title_cleaned"]

10        senior data architect
13          senior ai developer
14               data architect
16               data architect
17               data architect
                 ...           
735    chief technology officer
736    chief technology officer
737    chief technology officer
738    chief technology officer
739    chief technology officer
Name: title_cleaned, Length: 421, dtype: object

In [71]:
# Loading the JSON file back into a dictionary
import json
Job_progression_dictionary_file_path = "/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/job_progression_dictionary.json"

with open(Job_progression_dictionary_file_path, 'r') as json_file:
    sorted_full_job_progression_dict_lower = json.load(json_file)

print(sorted_full_job_progression_dict_lower)
# Output: {'name': 'John', 'age': 30, 'city': 'New York'}


{'ai architect': ['ai project manager', 'head of ai', 'chief ai officer'], 'ai developer': ['senior ai developer', 'ai architect', 'ai project manager', 'head of ai', 'chief ai officer'], 'ai project manager': ['head of ai', 'chief ai officer'], 'big data architect': ['head of big data', 'director of data engineering', 'chief information officer'], 'big data engineer': ['senior big data engineer', 'big data architect', 'head of big data', 'director of data engineering', 'chief information officer'], 'business analysis manager': ['business intelligence analyst', 'director of business analysis', 'chief strategy officer', 'data analyst', 'data scientist'], 'business analyst': ['senior business analyst', 'business analysis manager', 'business intelligence analyst', 'director of business analysis', 'chief strategy officer', 'data analyst', 'data scientist'], 'business intelligence analyst': ['senior business intelligence analyst', 'business intelligence manager', 'director of business intel

In [75]:
import pandas as pd
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ast
# Assuming you have the following data loaded into pandas DataFrames
# jobs_df = pd.DataFrame({'Job': ['Data Analyst', 'Data Scientist', ...], 'Skills': [['SQL', 'Python'], ['Machine Learning', 'Python', 'SQL'], ...]})
# courses_df = pd.DataFrame({'Course': ['Intro to Python', 'Advanced Machine Learning', ...], 'Skills_Taught': [['Python'], ['Machine Learning', 'Deep Learning'], ...]})
# skills_df = pd.DataFrame({'Skill': ['Python', 'SQL', 'Machine Learning', ...]})
# A simple function to select a random job and its required skills
def select_random_job(jobs_df):
    random_job = jobs_df.sample()
    return random_job.title_cleaned.values[0], random_job.skills_tagged.values[0]

def select_random_job(jobs_df):
    random_idx = random.randint(0, len(jobs_df) - 1)
    random_job = jobs_df.iloc[random_idx]
    return random_job.title_cleaned, random_job.skills_tagged


def select_next_job(current_job, jobs_df, job_progression_dict):
    next_job,next_skills,random_job_selection_flag="","",0
    # If the current job is in the progression dict, select the next job from there
    if current_job in job_progression_dict:
        next_jobs = job_progression_dict[current_job]
        next_jobs=list(set(jobs_df["title_cleaned"].unique()).intersection(set(next_jobs)))
        if len(next_jobs)!=0:
            next_job = random.choice(next_jobs)
            #print(next_job, " choosen")
            try:
                #next_skills = jobs_df[jobs_df['title_cleaned'] == next_job]['skills_tagged'].values[0]
                next_skills_rows = jobs_df[jobs_df['title_cleaned'] == next_job]
                if not next_skills_rows.empty:
                    random_row = next_skills_rows.sample(n=1)
                    next_skills = random_row['skills_tagged'].values[0]
            except:
                next_job = random.choice(next_jobs)
                #next_skills = jobs_df[jobs_df['title_cleaned'] == next_job]['skills_tagged'].values[0]
                next_skills_rows = jobs_df[jobs_df['title_cleaned'] == next_job]
                if not next_skills_rows.empty:
                    random_row = next_skills_rows.sample(n=1)
                    next_skills = random_row['skills_tagged'].values[0]
        else:
            # If not, fall back to random selection for this example
            next_job, next_skills = select_random_job(jobs_df)
            random_job_selection_flag=1
    else:
        next_job, next_skills = select_random_job(jobs_df)
        random_job_selection_flag=1
    return next_job, next_skills,random_job_selection_flag





# A simple function to select a course that teaches at least one of the required skills
def select_relevant_course(courses_df, required_skills):
    relevant_courses = courses_df[courses_df['Skills_Taught'].apply(lambda x: any(skill in x for skill in required_skills))]
    if not relevant_courses.empty:
        return relevant_courses.sample()
    return None


def calculate_course_score(course, skill_gap, weight_skill_match=1, weight_subscribers=0.001, weight_rating=0.5):
    # Calculate the proportion of skill gap covered by course
    skills_matched = sum(skill in course['skills_tagged'] for skill in skill_gap)
    skill_match_score = skills_matched / len(skill_gap) if skill_gap else 0

    # Factor in course popularity and rating
    #subscriber_score = course['num_subscribers'] * weight_subscribers if 'num_subscribers' in course else 0
    #rating_score = course['rating'] * weight_rating if 'rating' in course else 0

    # Calculate total score
    total_score = (skill_match_score * weight_skill_match) #+ subscriber_score + rating_score
    return total_score

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_c_cleaned["full_description"])

def calculate_similarity_score(skill_gap,df_c_cleaned):

    def preprocess_text(text):
        return " ".join(token.lemma_.lower() for token in nlp(text) if not token.is_punct and not token.is_stop)

    skill_gap_text = " ".join(skill_gap)
    # Vectorize the text

    skill_gap_vec = tfidf_vectorizer.transform([skill_gap_text])
    # Calculate cosine similarity
    cosine_similarities = cosine_similarity(skill_gap_vec, tfidf_matrix).flatten()
    df_c_cleaned["cosine_score"]=cosine_similarities
    return df_c_cleaned

def select_relevant_course(df_c_cleaned, skill_gap):
    # Apply the scoring function to each course
    df_c_cleaned['course_score'] = df_c_cleaned.apply(calculate_course_score, axis=1, skill_gap=skill_gap)
    df_c_cleaned = calculate_similarity_score(skill_gap,df_c_cleaned)
    # Sort courses based on the calculated score
    #df_c_cleaned = df_c_cleaned.sort_values(by='course_score', ascending=False)
    df_c_cleaned = df_c_cleaned.sort_values(by='cosine_score', ascending=False)
    # Select the top course if available
    top_course = df_c_cleaned.iloc[0] if not df_c_cleaned.empty else None
    return top_course

# Select the most relevant course based on the skill gap
#relevant_course = select_relevant_course(df_c_cleaned, skill_gap)
#relevant_course.head()

# Generate synthetic user data
def generate_synthetic_data(jobs_df, courses_df, skills_df, num_samples=10):
    synthetic_data = []
    for _ in range(num_samples):
        #select_random_job(jobs_df)
        current_job, current_skills = select_random_job(jobs_df)
        # Select a job to progress to that requires at least one additional skill
        next_job, next_skills,random_job_selection_flag = select_next_job(current_job,jobs_df,sorted_full_job_progression_dict_lower)
        #while set(next_skills).issubset(current_skills):
        #    next_job, next_skills = select_random_job(jobs_df)
        next_skills=ast.literal_eval(next_skills)
        current_skills=ast.literal_eval(current_skills)
        # Find the skill gap
        skill_gap = list(set(next_skills) - set(current_skills))
        #print(skill_gap)
        # Find a course that teaches one of the skills in the gap
        course = select_relevant_course(courses_df, skill_gap)
        #print(course)
        if course is not None:
            course_name = course.course_title
            course_skills = course.skills_tagged
            course_title = course.title
        else:
            # If no single course covers the gap, select a random one for this synthetic example
            course_name = "Random Course"
            course_skills = "Random Skill"
        
        # Append to the synthetic data list
        synthetic_data.append({
            'Current Job': current_job,
            'Next Job': next_job,
            'Current Skills': current_skills,
            'Next Skills': next_skills,
            'Skill Gap': skill_gap,
            'course title':course_title,
            'Course Taken': course_name,
            'Course Skills': course_skills,
            'random next job':random_job_selection_flag
        })
    
    return pd.DataFrame(synthetic_data)


# Generate synthetic user data
def generate_synthetic_next_job_data(jobs_df,courses_df,synthetic_user_data):
    synthetic_data = []
    for i,iterrow in synthetic_user_data.iterrows():
        #print(i, iterrow["Next Job"],iterrow["Next Skills"])
        #select_random_job(jobs_df)
        current_job, current_skills = iterrow["Next Job"],(iterrow["Next Skills"])
        # Select a job to progress to that requires at least one additional skill
        next_job, next_skills,random_job_selection_flag = select_next_job(current_job,jobs_df,sorted_full_job_progression_dict_lower)
        #while set(next_skills).issubset(current_skills):
        #    next_job, next_skills = select_random_job(jobs_df)
        next_skills=ast.literal_eval(next_skills)
        # Find the skill gap
        skill_gap = list(set(next_skills) - set(current_skills))
        #print(skill_gap)
        # Find a course that teaches one of the skills in the gap
        course = select_relevant_course(courses_df, skill_gap)
        #print(course)
        if course is not None:
            course_name = course.course_title
            course_skills = course.skills_tagged
            course_title = course.title
        else:
            # If no single course covers the gap, select a random one for this synthetic example
            course_name = "Random Course"
            course_skills = "Random Skill"
        
        # Append to the synthetic data list
        synthetic_data.append({
            'Current Job': current_job,
            'Next Job': next_job,
            'Current Skills': current_skills,
            'Next Skills': next_skills,
            'Skill Gap': skill_gap,
            'course title':course_title,
            'Course Taken': course_name,
            'Course Skills': course_skills,
            'random next job':random_job_selection_flag
        })
    
    return pd.DataFrame(synthetic_data)



# Generate the synthetic data
#synthetic_user_data_v1 = generate_synthetic_data(df_j_cleaned, df_c_cleaned, skills_list,10000)
#synthetic_user__next_data_v2 = generate_synthetic_next_job_data(df_j_cleaned, df_c_cleaned,synthetic_user_data_v1)
#synthetic_user__next_data_v3 = generate_synthetic_next_job_data(df_j_cleaned, df_c_cleaned,synthetic_user__next_data_v2)
#synthetic_user__next_data_v4 = generate_synthetic_next_job_data(df_j_cleaned, df_c_cleaned,synthetic_user__next_data_v3)
#synthetic_user_data = pd.concat([synthetic_user_data_v1,synthetic_user__next_data_v2,synthetic_user__next_data_v3,synthetic_user__next_data_v4],axis=0)
#print(synthetic_user_data.head())


In [92]:
synthetic_user_data_v1 = generate_synthetic_data(df_j_cleaned, df_c_cleaned, skills_list,10)
synthetic_user__next_data_v2 = generate_synthetic_next_job_data(df_j_cleaned, df_c_cleaned,synthetic_user_data_v1)
#synthetic_user__next_data_v3 = generate_synthetic_next_job_data(df_j_cleaned, df_c_cleaned,synthetic_user__next_data_v2)
#synthetic_user__next_data_v4 = generate_synthetic_next_job_data(df_j_cleaned, df_c_cleaned,synthetic_user__next_data_v3)
#synthetic_user_data = pd.concat([synthetic_user_data_v1,synthetic_user__next_data_v2,synthetic_user__next_data_v3,synthetic_user__next_data_v4],axis=0)
#synthetic_user_data.to_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/synthetic_data_half_cleaned_mac_full_v1.csv")
synthetic_user_data = pd.concat([synthetic_user_data_v1,synthetic_user__next_data_v2],axis=0)

In [94]:
#del synthetic_user_data["Unnamed: 0"]

In [95]:
synthetic_user_data

,Current Job,Next Job,Current Skills,Next Skills,Skill Gap,course title,Course Taken,Course Skills,random next job
0,data architect,director of analytics,"{analytics, business intelligence, nosql, data...","{analytics, data mining, quantitative analysis...","[data mining, quantitative analysis, data scie...",python machine learning data mining bootcamp,data mining,"{'machine learning algorithms', 'visualization...",0
1,senior research scientist,principal scientist,{computer science},"{problem solving, data integration, data visua...","[problem solving, data integration, data visua...",getting started data management,database management,"{'data integration', 'big data', 'python progr...",0
2,machine learning engineer,machine learning architect,"{scala, statistical analysis, data analysis, c...","{apache spark, computer vision, natural langua...","[apache spark, nlp, natural language processin...",NaN,docker,set(),0
3,senior data product manager,head of big data,"{technology solutions, problem solving, build ...","{data storage, data management, analytics, enc...","[business intelligence, data engineering, data...",intro data data science,data science,"{'data structures', 'data science'}",0
4,ai project manager,database administrator,"{project management, machine learning, agile p...",{data manipulation},[data manipulation],data manipulation python pandas r sql alteryx,undefined,"{'data science', 'big data', 'data manipulatio...",1
5,data scientist,senior data scientist,"{scala, problem solving, probability, data min...","{scala, analytics, data analysis, pyspark, dat...","[data analysis, analytics, pyspark, data minin...",learning predictive analytics python,python,"{'data science', 'analytics', 'machine learnin...",0
6,data scientist,data science manager,"{analytics, data engineering, sql, project man...","{analytics, analytical techniques, sql, python...","[analytical techniques, data selection, busine...",data analytics crash course,data analysis,"{'analytics', 'statistical methods', 'big data...",0
7,data analyst,chief information officer,"{google analytics, statistical analysis, analy...","{project management, technology solutions, dat...","[project management, technology solutions, dat...",cdmp metadata specialist exam questions practi...,undefined,"{'data integration', 'data architecture', 'big...",0
8,senior data engineer,data architect,"{analytics, data engineering, data domain, gen...","{business intelligence, data processing, data ...","[business intelligence, data processing, data ...",data science fundamentals,data science,"{'computer science', 'data science', 'algorith...",0
9,director of analytics,data engineer,"{analytics, data science, computer science, co...","{analytics, pyspark, data engineering, sql, az...","[pyspark, data engineering, sql, azure data fa...",azure data factory data engineering cloud,azure synapse analytics,"{'data integration', 'big data analytics', 'az...",1


In [81]:
#synthetic_user_data_v1 = pd.read_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/synthetic_data_half_cleaned_mac_v1.csv")



0 director of data science {'analytics', 'sql', 'business requirements', 'predictive modeling', 'algorithms', 'data quality', 'ad hoc analysis', 'data governance', 'data science', 'dashboard'}
1 director of data engineering {'b testing', 'product analytics'}
chief technology officer  choosen
2 senior machine learning engineer {'problem solving', 'data structures', 'conversational ai', 'generative ai', 'natural language processing', 'python', 'product analysis', 'tensorflow', 'docker', 'algorithms', 'pytorch', 'computer science', 'machine learning', 'artificial intelligence', 'llm', 'c', 'nlp', 'chatgpt'}
machine learning architect  choosen
3 director of product management {'project management'}
director of data engineering  choosen
4 principal data scientist {'statistical analysis', 'problem solving', 'data science', 'encryption', 'storage virtualization', 'statistical', 'computer science', 'statistical modeling', 'machine learning'}
director of data science  choosen
5 director of anal

,Current Job,Next Job,Current Skills,Next Skills,Skill Gap,course title,Course Taken,Course Skills,random next job
0,director of data science,senior data architect,"{analytics, sql, business requirements, predic...","{data analysis, analytics, pl sql, linux scrip...","[data analysis, linux scripting, problem solvi...",data visualization python masterclass python z,data science,"{'visualization', 'sql', 'dataset', 'statistic...",1
1,director of data engineering,chief technology officer,"{b testing, product analytics}","{project management, problem solving, technolo...","[project management, problem solving, technolo...",cdmp metadata specialist exam questions practi...,undefined,"{'data integration', 'data architecture', 'big...",0
2,senior machine learning engineer,machine learning architect,"{problem solving, data structures, conversatio...","{apache spark, computer vision, natural langua...","[apache spark, computer vision, big data, deep...",deep learning apache spark masterclass,deep learning,"{'deep learning', 'algorithms', 'apache spark'...",0
3,director of product management,director of data engineering,{project management},"{scala, data processing, data engineering, pyt...","[scala, data processing, data engineering, pyt...",apache airflow complete hands beginner advance...,apache airflow,"{'data engineering', 'airflow', 'apache airflo...",0
4,principal data scientist,director of data science,"{statistical analysis, problem solving, data s...","{analytics, data mining, quantitative analysis...","[analytics, data mining, quantitative analysis...",data mining r beginner advanced,data mining,"{'data visualization', 'analytics', 'visualiza...",0
5,director of analytics,data scientist,"{data science, analytics, workforce management...","{data analysis, data engineering, python, data...","[data analysis, machine learning, data enginee...",machine learning project heart attack predicti...,data analysis,"{'r programming', 'nosql', 'python programming...",1
6,enterprise architect,chief information officer,"{mysql, data retention, data management, data ...","{http, data driven decision making}","[http, data driven decision making]",python,python,"{'http', 'python'}",0
7,director of analytics,director of product management,"{analytics, data science, computer science, co...",{project management},[project management],cdmp metadata specialist exam questions practi...,undefined,"{'data integration', 'data architecture', 'big...",1
8,data analyst,enterprise architect,"{microsoft office, data synchronization, data ...","{mysql, data retention, data management, data ...","[mysql, data retention, big data, data managem...",intro data data science,data science,"{'data structures', 'data science'}",0
9,director of product management,chief information officer,{analytics},{project management},[project management],cdmp metadata specialist exam questions practi...,undefined,"{'data integration', 'data architecture', 'big...",0


In [62]:
import ast 
df = synthetic_user_data.copy()
# Fill missing values if necessary
df.fillna({'Course Taken': 'no course','course title': 'no course'}, inplace=True)
df["Course Taken"]=df["Course Taken"].apply(lambda x:x.replace("NAN","No Course"))

df["Course Skills"]=df["Course Skills"].apply(lambda x:ast.literal_eval(x))
df["Course Skills"].apply(lambda x: x if len(x)!=0 else "no skills")
#df["Course title"]=df["Course title"].apply(lambda x:x.replace("NAN","No Course"))
def set_to_string(skill_set):
    if not skill_set:  # Checks if the set is empty
        return 'no skills' 
    # Check if skill_set is a string that needs to be evaluated
    if isinstance(skill_set, str):
        try:
            # Try to evaluate the string as a set
            skill_set = ast.literal_eval(skill_set)
        except (ValueError, SyntaxError):
            # Handle cases where the string is not a valid set
            pass  # You might want to return a default value or handle this case as needed
    # Convert to string if it's a set or list
    if isinstance(skill_set, (set, list)):
        return ', '.join(skill_set)
    return skill_set 

set_columns = ['Current Skills', 'Next Skills', 'Skill Gap', 'Course Skills']
for col in set_columns:
    print(col)
    #df[col]=df[col].apply(lambda x:ast.literal_eval(x))
    df[col] = df[col].apply(set_to_string)





Current Skills
Next Skills
Skill Gap
Course Skills


In [63]:
df.to_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/synthetic_data_cleaned_v2.csv")


In [64]:
df

,Current Job,Next Job,Current Skills,Next Skills,Skill Gap,course title,Course Taken,Course Skills,random next job
0,chief technology officer,data architect,"analytics, business requirements, predictive m...","c, data virtualization, analytics, pyspark, da...","c, data virtualization, pyspark, data structur...",intro data data science,data science,"data science, data structures",1
1,chief technology officer,data scientist,"analytics, cloud computing, computer science, ...","analytics, data engineering, sql, project mana...","data engineering, sql, project management, pyt...",probability statistics data science,data science,"analytics, data engineering, probability stati...",1
2,data science manager,director of data science,"data analysis, analytics, sql, project managem...","analytics, sql, business requirements, predict...","business requirements, data quality, predictiv...",predictive analytics modeling python,undefined,"data analysis, data mining, visualization, pyt...",0
3,data analyst,senior data product manager,"data analysis, analytics, computer programming...","data analysis, analytics, data engineering, ma...","data engineering, machine learning, statistica...",statistics statistical modeling explained python,data analysis,"data analysis, statistical analysis, python, p...",0
4,chief technology officer,statistician,"computer science, data processing, data integr...","data processing, prompts, http, data science, ...","prompts, http, data science, statistical, c, s...",statistics statistical modeling explained python,data analysis,"data analysis, statistical analysis, python, p...",1
5,senior business analyst,data scientist,"microsoft office, sql, project management, agi...","computer science, generative ai, data security...","generative ai, data security, data normalizati...",introduction data science,data science,"data analysis, analytics, sql, python, busines...",0
6,database administrator,chief information officer,"problem solving, database servers, sql","computer science, problem solving",computer science,basics computer science information systems bcsis,information systems,"computer science, data science, data management",0
7,director of analytics,data scientist,"analytics, data science, computer science, com...","data analysis, problem solving, computer scien...","data analysis, problem solving, sql, project m...",python sql,sql,"sqlite, python, sql",1
8,director of data science,database administrator,"analytics, data mining, quantitative analysis,...","database administration, sql","database administration, sql",building bulk sql queries excel sql oracle toad,sql,"database management, data mining, sql",1
9,data engineer,data manager,"analytics, cloud computing, problem solving, d...",no skills,no skills,machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",0


In [14]:
# convert text to numeric using tfidf [not used currently]

import spacy

# Load SpaCy model
nlp = spacy.load('en_core_web_md')

def preprocess_text(text):
    doc = nlp(text)
    return " ".join(token.lower_ for token in doc if not token.is_punct and not token.is_stop)

text_columns = ['Current Job', 'Next Job', 'Current Skills', 'Next Skills', 'Skill Gap', 'course title', 'Course Skills','Course Taken']
for col in text_columns:
    print(col)
    #df[col] = df[col].apply(preprocess_text)

from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Assuming df is your DataFrame and text_columns are the columns to vectorize
tfidf_vectorizer = TfidfVectorizer()

# Create a DataFrame to hold all TF-IDF features
tfidf_df = pd.DataFrame()

for col in ["course title"]:
    # Apply TF-IDF transformation
    tfidf_matrix = tfidf_vectorizer.fit_transform(df[col])

    # Convert to dense format and create a DataFrame
    tfidf_dense = pd.DataFrame(tfidf_matrix.todense(), columns=tfidf_vectorizer.get_feature_names_out())

    # Prefix column names with the original column name for clarity
    tfidf_dense.columns = [f"{col}_{feature}" for feature in tfidf_dense.columns]

    # Concatenate with the main TF-IDF DataFrame
    tfidf_df = pd.concat([tfidf_df, tfidf_dense], axis=1)

# Combine the original DataFrame with the TF-IDF features DataFrame
final_df = pd.concat([df, tfidf_df], axis=1)

Current Job
Next Job
Current Skills
Next Skills
Skill Gap
course title
Course Skills
Course Taken


In [15]:
final_df

,Current Job,Next Job,Current Skills,Next Skills,Skill Gap,course title,Course Taken,Course Skills,random next job,course title_ai,course title_bonus,course title_chatgpt,course title_learning,course title_machine,course title_python
0,ai project manager,data scientist,"project management, problem solving, microsoft...","analytics, sql, python, algorithms, data stora...","d, h, u, q, y",machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",1,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
1,product analyst,data product manager,,"project management, analytics, agile software ...","w, ', v, o, }, {, , g, b, c, a, j, m, ,, p, d...",machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",0,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
2,data analyst,director of data engineering,"microsoft office, data synchronization, data m...","computer science, test data management, devops","u, v",machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",0,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
3,unmatched,database administrator,"problem solving, data collection","database administration, sql",q,machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",1,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
4,senior of data science,statistician,"data analysis, analytics, business intelligenc...","data analysis, statistical analysis, data mana...",no skills,machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",1,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
5,unmatched,machine learning engineer,"problem solving, project management, data mana...","data mining, bigtable, mapreduce, python, comp...","h, u, f",machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",1,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
6,director of product management,chief product officer,"technology solutions, problem solving, risk an...","analytics, data driven decision making, produc...","d, f",machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",0,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
7,business analyst,business intelligence analyst,"business analysis, qualitative analysis, sql","analytics, business intelligence","g, c",machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",0,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
8,business intelligence manager,director of business intelligence,"data analysis, analytics, problem solving, bus...","data analysis, analytics, business intelligenc...","k, f, w",machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",0,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
9,director of analytics,unmatched,"analytics, data science, computer science, com...","data structures, computer science, data integr...",k,machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",1,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
